## General requirements & preparation

This timeseries script uses functions from atlite to generate weather timeseries for GENeSYS-MOD. It uses the ERA5 weather dataset, which is downloaded and transformed locally.
Please be aware that for the entire European dataset, a large amount of system memory is required (at least 24GB of RAM).


To use this script, you need to have the following packages installed:
- numpy
- matplotlib
- seaborn
- pandas / geopandas
- scikit-learn
- cartopy
- xarray
- atlite

In addition, in order to be able to load the ERA5 cutouts, you need an API key from https://cds.climate.copernicus.eu/user/register?destination=%2F%23!%2Fhome

To activate the API key, follow the instructions at https://cds.climate.copernicus.eu/api-how-to

For more information on how to run atlite, please refer to https://atlite.readthedocs.io/en/latest/

In [ ]:
from functions import *

In [ ]:
onshore_turbine = 'Vestas_V112_3MW'
offshore_turbine = 'Vestas_V164_7MW_offshore'
solar_panel = 'CSi'
geo_file = "geodata/natural_earth_world_admin1.geojson"


timeframe = "2018-01-01"   # timeframe for the timeseries. format example: 2018 OR 2018-01 OR 2018-01-01
filename = "germany"   # filename for the downloaded raw data (cutout). this needs to be changed for every new run/year/region
output_dir = create_output_folder(timeframe)   # creates an output folder, if it does not exist yet

# choose either 0 for country-level (NUTS-0) or 1 for major socio-economic regions (NUTS-1)
admin = 0

# country ISO 3166-1 alpha-2 code. If all regions of cutout are needed, leave list empty
regions = ["DE"] 

# cutout cooridnates. Leave empty, if the cutout should be taken from a shapefile (with above specified regions)
cutout_north_west = []
cutout_south_east = []

In [3]:
# Here, you need to define the PV slope and azimuth of the panels (defaults to slope of 36.7 and azimuth of 180 if not defined)
pv_slope = 36.7
pv_azimuth = 180

## Region Defitinition & Cutout preparation

In this block, you need to define your coordinates, either by using your own fixed coordinates (by defining custom bounds), or using the automatic .geojson mapping for the country of your choice.

Be aware that the first preparation of the cutout will take significant time!

In [ ]:
cutout = get_cutout(filename, 
                    timeframe, 
                    regions=regions, 
                    dx=0.25, 
                    dy=0.25)

Plotting your cutout to ensure that everything is correct!

You can add zoom=True/False to zoom into your selection and use size to set the size of the graph if needed.

In [ ]:
plot_country_map(cutout) # plot_country_map(cutout,zoom=False,size=8)

## Functions for timeseries generation

In [6]:
coords_onshore, coords_offshore = get_coords(cutout, 
                                             regions, 
                                             geo_file, 
                                             admin=admin)

In [7]:
pv_inf, pv_avg, pv_opt = pv_capacity_factors(cutout, 
                                             coords_onshore, 
                                             solar_panel,
                                             pv_slope=pv_slope,
                                             pv_azimuth=pv_azimuth, 
                                             timeframe=timeframe, 
                                             filename=filename,
                                             write_raw_data=False, # determines if you want to have the aggregated outputs per administrative region or the full dataset of each individual coordinate (default: False)
                                             output_dir=output_dir)

In [8]:
pv_tra = pv_capacity_factors(cutout, 
                             coords_onshore, 
                             solar_panel,
                             tracking= "horizontal",
                             pv_azimuth=0, #refers to oriantation of the axis in this case
                             timeframe=timeframe, 
                             filename=filename,
                             write_raw_data=False,
                             output_dir=output_dir)

In [ ]:
display(pv_inf.mean())
display(pv_avg.mean())
display(pv_opt.mean())

In [ ]:
display(pv_tra.mean())

In [11]:
wind_onshore_inf, wind_onshore_avg, wind_onshore_opt = wind_onshore_capacity_factors(cutout, 
                                                                                     coords_onshore, 
                                                                                     onshore_turbine,  
                                                                                     timeframe=timeframe, 
                                                                                     filename=filename,
                                                                                     write_raw_data=False, # determines if you want to have the aggregated outputs per administrative region or the full dataset of each individual coordinate (default: False)
                                                                                     output_dir=output_dir)

In [ ]:
display(wind_onshore_inf.mean()) 
display(wind_onshore_avg.mean()) 
display(wind_onshore_opt.mean())

In [13]:
wind_offshore_shallow, wind_offshore_transitional, wind_offshore_deep = wind_offshore_capacity_factors(cutout, 
                                                                                                       coords_offshore, 
                                                                                                       offshore_turbine, 
                                                                                                       timeframe=timeframe,
                                                                                                       filename=filename,
                                                                                                       write_raw_data=False, # determines if you want to have the aggregated outputs per administrative region or the full dataset of each individual coordinate (default: False)
                                                                                                       output_dir=output_dir)

In [ ]:
display(wind_offshore_shallow.mean())
display(wind_offshore_transitional.mean())
display(wind_offshore_deep.mean())

In [15]:
df_heatpump_ground_cop, df_cooling, df_heating, df_heatpump_cop = temperature_timeseries(cutout,
                                                                                        coords_onshore,
                                                                                        timeframe=timeframe,
                                                                                        filename=filename,
                                                                                        write_raw_data=False, # determines if you want to have the aggregated outputs per administrative region or the full dataset of each individual coordinate (default: False)
                                                                                        output_dir=output_dir)

In [ ]:
display(df_heatpump_ground_cop.mean())
display(df_heatpump_cop.mean())
display(df_cooling.mean())
display(df_heating.mean())

### GIS-based renewable energy potentials

##### This part is still somewhat experimental. It uses functionality from the Atlite package to calculate available capcities for utility-scale PV, onshore wind, and rooftop PV installations.
##### It uses the information from the timeseries part to compute the shapefiles, the only thing that needs to be provided is the excluder zones (e.g. WDPA for protected areas). CORINE is provided for land cover in Europe, anything else would need to be separately downloaded.

In [ ]:
shapes, regions_name_en = gis_get_country_geometry(regions,admin,cutout)

In [18]:
CORINE = "geodata/corine.tif"
WDPA = "geodata/WDPA_Oct2024_Public_shp-polygons.shp"

excluder = ExclusionContainer()
excluder.add_raster(CORINE, codes=range(20))
#excluder.add_geometry(WDPA)

cities = ExclusionContainer()
cities.add_raster(CORINE, codes=range(1,6),invert=True)

In [ ]:
AvailabilityMatrix = calculate_and_plot_available_area(admin,cutout,shapes,regions_name_en,excluder)

In [ ]:
AvailabilityMatrix_Rooftop = calculate_and_plot_available_rooftops(admin,cutout,shapes,regions_name_en,cities)

In [21]:
pv_cap_per_sqkm = 100                   # MW
pv_percent_land_available = 0.03        # % of suitable area
wind_cap_per_sqkm = 27                  # MW
wind_percent_land_available = 0.03      # % of suitable area
rooftop_cap_per_sqkm = 100              # MW
rooftop_percent_area_available = 0.2    # % of all rooftops

In [22]:
output_df = calculate_capacity_potentials(cutout,coords_onshore,AvailabilityMatrix,AvailabilityMatrix_Rooftop,pv_cap_per_sqkm,pv_percent_land_available,wind_cap_per_sqkm,wind_percent_land_available,rooftop_cap_per_sqkm,rooftop_percent_area_available)

In [ ]:
# here is the resulting DataFrame containing all the potentials and areas that have been calculated
output_df